# Use the Notebook to build the code to scrape the Wikipedia page
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [1]:
import pandas as pd
url='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=942851379'

df=pd.read_html(url, header=0)[0]

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [2]:
df_drop=df[df['Borough']!='Not assigned']

In [3]:
df_final=(df_drop.groupby(['Postcode','Borough'])
   .agg({'Neighbourhood' : ', '.join})
   .reset_index()
   .reindex(columns=df.columns))
df_final

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [4]:
df_final.shape

(103, 3)

# Get the geographical coordinates of the neighborhoods in the Toronto.

In [5]:
locdata = pd.read_csv("http://cocl.us/Geospatial_data")
locdata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
df_merge=pd.merge(df_final, locdata, how='inner', left_on='Postcode', right_on='Postal Code')
df_merge = df_merge.drop('Postal Code', 1)
df_merge.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Explore and cluster the neighborhoods in Toronto. 

In [7]:
import geopy
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import sklearn
import json
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [8]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# Creating a map of Toronto with neigborhoods using latitude and longitude values

In [10]:

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
Toronto_geo = df_merge
# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_geo['Latitude'], Toronto_geo['Longitude'], Toronto_geo['Borough'], Toronto_geo['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_Toronto)  
    
map_Toronto

# Replicate the same analysis we did to the New York City data to Toronto

In [11]:
CLIENT_ID = '5DHWS1YO2QADABIXLXTJPXV1MBQLNQLPISX4CBEMIP1KNKLR' # your Foursquare ID
CLIENT_SECRET = 'O4ODQCYFCNH2DW2WI52PE5NUN23LNSU5UUHB5Q4BLH3S1WMR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5DHWS1YO2QADABIXLXTJPXV1MBQLNQLPISX4CBEMIP1KNKLR
CLIENT_SECRET:O4ODQCYFCNH2DW2WI52PE5NUN23LNSU5UUHB5Q4BLH3S1WMR


# Let's explore the first neighborhood in our dataframe.

In [13]:
Toronto_geo.loc[0, 'Neighbourhood']

'Rouge, Malvern'

# Get the neighborhood's latitude and longitude values.

In [14]:
neighborhood_latitude = Toronto_geo.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_geo.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_geo.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


# Now, let's get the top 20 venues that are in Rouge Malvern within a radius of 500 meters

In [26]:
LIMIT = 20 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=5DHWS1YO2QADABIXLXTJPXV1MBQLNQLPISX4CBEMIP1KNKLR&client_secret=O4ODQCYFCNH2DW2WI52PE5NUN23LNSU5UUHB5Q4BLH3S1WMR&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=20'

# Send the GET request and examine the results

In [17]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e80a61b0f5968001b7d9bcf'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Now we are ready to clean the json and structure it into a pandas dataframe

In [29]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


# Explore all Neighbourhoods in Toronto

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues

In [32]:
toronto_venues = getNearbyVenues(names=Toronto_geo['Neighbourhood'],
                                   latitudes=Toronto_geo['Latitude'],
                                   longitudes=Toronto_geo['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

# Let's check the size of the resulting dataframe

In [33]:
print(toronto_venues.shape)
toronto_venues.head()

(1069, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


# Let's check how many venues were returned for each neighborhood

In [34]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",20,20,20,20,20,20
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",8,8,8,8,8,8
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Berczy Park,20,20,20,20,20,20


# Let's find out how many unique categories can be curated from all the returned venues

In [35]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 207 uniques categories.


 # Analyze Each Neighborhood

In [36]:

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# And let's examine the new dataframe size.

In [37]:
toronto_onehot.shape

(1069, 208)

# let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [38]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,...,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


# Let's confirm the new size

In [39]:
toronto_grouped.shape

(100, 208)

# Let's print each neighborhood along with the top 5 most common venues

In [40]:

num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']: 
    print("----+hood+----") 
    temp = toronto_grouped[toronto_grouped['Neighbourhood']==hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----+hood+----
              venue  freq
0             Hotel  0.10
1  Asian Restaurant  0.10
2             Plaza  0.05
3        Steakhouse  0.05
4        Restaurant  0.05


----+hood+----
                       venue  freq
0  Latin American Restaurant  0.25
1             Clothing Store  0.25
2             Breakfast Spot  0.25
3                     Lounge  0.25
4          Accessories Store  0.00


----+hood+----
                        venue  freq
0                  Playground   0.5
1                        Park   0.5
2           Accessories Store   0.0
3  Modern European Restaurant   0.0
4                      Lounge   0.0


----+hood+----
                  venue  freq
0         Grocery Store  0.25
1           Pizza Place  0.12
2   Fried Chicken Joint  0.12
3        Sandwich Place  0.12
4  Fast Food Restaurant  0.12


----+hood+----
                venue  freq
0         Pizza Place  0.22
1                 Gym  0.11
2  Athletics & Sports  0.11
3         Coffee Shop  0.11
4              

# Let's put that into a pandas dataframe

In [43]:
import numpy as np

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Hotel,Asian Restaurant,Neighborhood,Gym / Fitness Center,Restaurant,Seafood Restaurant,Coffee Shop,Lounge,Speakeasy,Steakhouse
1,Agincourt,Clothing Store,Lounge,Breakfast Spot,Latin American Restaurant,Yoga Studio,Department Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Dance Studio,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pharmacy,Pizza Place,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Yoga Studio,Deli / Bodega,Dog Run
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pub,Skating Rink,Athletics & Sports,Sandwich Place,Pharmacy,Dance Studio,Diner


# Cluster Neighborhoods

In [45]:

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 4, 0, 0, 0, 3, 3, 3, 3], dtype=int32)

# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [46]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_.astype(int))

toronto_merged = Toronto_geo

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3.0,Bar,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3.0,Rental Car Location,Breakfast Spot,Spa,Intersection,Bank,Medical Center,Mexican Restaurant,Electronics Store,Yoga Studio,Drugstore
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Soccer Field,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3.0,Bank,Fried Chicken Joint,Caribbean Restaurant,Gas Station,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Department Store,Dessert Shop


# Finally, let's visualize the resulting clusters

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

toronto_merged = toronto_merged.dropna(subset=['Cluster Labels'])

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

## Cluster 1

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0.0,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner
3,Scarborough,0.0,Coffee Shop,Korean Restaurant,Soccer Field,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center
6,Scarborough,0.0,Coffee Shop,Department Store,Discount Store,Bus Station,Hobby Shop,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center
13,Scarborough,0.0,Pizza Place,Noodle House,Italian Restaurant,Intersection,Rental Car Location,Fast Food Restaurant,Bank,Thai Restaurant,Chinese Restaurant,Shopping Mall
15,Scarborough,0.0,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Pizza Place,Electronics Store,Bubble Tea Shop,Bank,Supermarket,Breakfast Spot,Cosmetics Shop
24,North York,0.0,Coffee Shop,Pharmacy,Grocery Store,Pizza Place,Discount Store,Bank,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run
28,North York,0.0,Coffee Shop,Bank,Sandwich Place,Pharmacy,Deli / Bodega,Sushi Restaurant,Ice Cream Shop,Middle Eastern Restaurant,Restaurant,Pizza Place
33,North York,0.0,Coffee Shop,Liquor Store,Grocery Store,Discount Store,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Diner
34,North York,0.0,Hockey Arena,Coffee Shop,Pizza Place,Portuguese Restaurant,Intersection,Dance Studio,Drugstore,Dog Run,Distribution Center,Discount Store
35,East York,0.0,Pizza Place,Intersection,Gastropub,Breakfast Spot,Pharmacy,Gym / Fitness Center,Bank,Fast Food Restaurant,Bus Line,Athletics & Sports


## Cluster 2

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,North York,1.0,Food Truck,Baseball Field,Yoga Studio,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner
91,Etobicoke,1.0,Baseball Field,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner
97,North York,1.0,Baseball Field,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner


## Cluster 3

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,2.0,Garden,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner


## Cluster 4

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3.0,Bar,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner
2,Scarborough,3.0,Rental Car Location,Breakfast Spot,Spa,Intersection,Bank,Medical Center,Mexican Restaurant,Electronics Store,Yoga Studio,Drugstore
4,Scarborough,3.0,Bank,Fried Chicken Joint,Caribbean Restaurant,Gas Station,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Department Store,Dessert Shop
7,Scarborough,3.0,Bakery,Bus Line,Soccer Field,Metro Station,Bus Station,Intersection,Ice Cream Shop,Park,Fruit & Vegetable Store,Deli / Bodega
8,Scarborough,3.0,American Restaurant,Movie Theater,Motel,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store
9,Scarborough,3.0,College Stadium,General Entertainment,Skating Rink,Café,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center
10,Scarborough,3.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store
11,Scarborough,3.0,Auto Garage,Shopping Mall,Breakfast Spot,Vietnamese Restaurant,Sandwich Place,Middle Eastern Restaurant,Bakery,Department Store,Drugstore,Dog Run
12,Scarborough,3.0,Clothing Store,Lounge,Breakfast Spot,Latin American Restaurant,Yoga Studio,Department Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center
17,North York,3.0,Golf Course,Dog Run,Fast Food Restaurant,Pool,Mediterranean Restaurant,Deli / Bodega,Eastern European Restaurant,Drugstore,Distribution Center,Discount Store


## Cluster 5

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,4.0,Playground,Convenience Store,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store
14,Scarborough,4.0,Park,Playground,Dance Studio,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
23,North York,4.0,Park,Convenience Store,Bar,Bank,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center
25,North York,4.0,Park,Food & Drink Shop,BBQ Joint,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store
30,North York,4.0,Park,Airport,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner
40,East York,4.0,Coffee Shop,Park,Convenience Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store
44,Central Toronto,4.0,Park,Swim School,Bus Line,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner
48,Central Toronto,4.0,Playground,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner
50,Downtown Toronto,4.0,Park,Playground,Trail,Dance Studio,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner
64,Central Toronto,4.0,Jewelry Store,Park,Trail,Sushi Restaurant,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store
